In [8]:
# Define Neo4j connections
from neo4j import GraphDatabase
import pandas as pd

host = 'bolt://localhost:7687'
user = 'neo4j'
password = 'letmein'
driver = GraphDatabase.driver(host,auth=(user, password))
                                         

def run_query(query, params={}):
    with driver.session() as session:
        result = session.run(query, params)
        return pd.DataFrame([r.values() for r in result], columns=result.keys())

In [18]:
data = run_query("""
MATCH (s)-[r]->(t)
RETURN toString(id(s)) as source, toString(id(t)) AS target, type(r) as type
""")

In [19]:
data.head()

,source,target,type
0,0,12590,interacts
1,0,8752,interacts
2,0,7915,interacts
3,0,21711,interacts
4,0,6447,interacts


In [22]:
from pykeen.triples import TriplesFactory


tf = TriplesFactory.from_labeled_triples(
  data[["source", "type", "target"]].values,
  create_inverse_triples=False,
  entity_to_id=None,
  relation_to_id=None,
  compact_id=True,
  filter_out_candidate_inverse_relations=True,
  metadata=None,
)

In [23]:
training, testing, validation = tf.split([.8, .1, .1])

using automatically assigned random_state=655788516


In [ ]:
from pykeen.pipeline import pipeline

result = pipeline(
    training=training,
    testing=testing,
    validation=validation,
    model='RotatE',
    stopper='early',
    epochs=20,  

)

Training epochs on cpu:   0%|          | 0/20 [00:00<?, ?epoch/s]

Training batches on cpu:   0%|          | 0/1756 [00:00<?, ?batch/s]

In [ ]:
from pykeen.models import predict

# Score top K triples
top_k_predictions_df = predict.get_all_prediction_df(result.model, k=150, triples_factory=training)

In [ ]:
top_k_predictions_df[top_k_predictions_df['relation_label'] == 'treats']